In [1]:
import numpy as np
import matplotlib.pyplot as plt
import requests
from bs4 import BeautifulSoup
from IPython.display import Markdown, display,IFrame,HTML,Image

from scipy.optimize import minimize
from scipy.integrate import quad

In [2]:
url='https://www.janestreet.com/puzzles/robot-capture-the-flag-index/'
res = requests.get(url)
soup = BeautifulSoup(res.content, 'html.parser')
y =[text.replace(">","greater than") for text in soup.body.stripped_strings]

#display([(i,j) for i,j in enumerate(y)])
display(Markdown("### "+y[13]+"\n"+"\n".join(y[16:47])))

### Robot Capture-the-Flag
It’s been a while and change, but the
Robot
Games
are back once again. This time it’s
Capture the Flag
!
Two robots, Aaron and Erin, have made it to this year’s final! Initially they are situated at the center of a unit circle. A
flag
is placed somewhere inside the circle, at a location chosen uniformly at random. Once the flag is placed, Aaron is able to deduce its
distance
to the flag, and Erin is only able to deduce its
direction
to the flag. (Equivalently: if (
r
,
θ
) are the polar coordinates of the flag’s location, Aaron is told
r
and Erin is told
θ
.)
Both robots are allowed to make a
single move
after the flag is placed, if they wish. Any move they make is without knowledge of what the other robot is doing. (And they may not move outside the circle.)
Whichever robot is
closer
to the flag after these moves
captures
the flag and is declared the winner!
During the preliminaries it was discovered that Erin is programmed to play a fixed distance along the detected angle
θ

In [3]:
# Erin strategy => jump x in direction of theta
# Aaron strategy stay at center if r < x/2  otherwise jump a distance y that maximises the chance of winning 
# Erin minimises the chance of an aaron win

theta = lambda r, x: np.arcsin(np.abs(r-x)/r)/np.pi * 2 * r   
prob_aron_win = lambda x: (x/2)**2 + quad(theta, x/2, 1,args=(x,))[0]  

solved = minimize(prob_aron_win, x0=0.5, bounds=[(0,1)])

display(Markdown("## Erin's best strategy is to jump *{:.10f}* giving Aaron a *{:.10f}* chance of winning".format(solved['x'][0],solved['fun'])))

## Erin's best strategy is to jump *0.5013064106* giving Aaron a *0.1661864865* chance of winning

In [4]:
url='https://www.janestreet.com/puzzles/robot-capture-the-flag-solution/'
res = requests.get(url)
soup = BeautifulSoup(res.content, 'html.parser')
y =[text for text in soup.body.stripped_strings]

#display([(i,j) for i,j in enumerate(y)])
display(Markdown("### "+y[13]+"\n"+"\n".join(y[16:47])))

### Robot Capture-the-Flag
Aaron has learned that Erin always plays a fixed distance — call it
e
— along the correct angle. Erin knows that Aaron knows this, and that Aaron will pick a distance
a
(and an angle at random, presumably) to maximize
P
(Aaron wins). So Erin should pick a value for
e
that minimizes this probability.
So we need to determine
P
(Aaron wins) for given values of
e
and
r
. If
r
is less than
e
/2, Aaron should choose to stay at the center, thus guaranteeing a win. Otherwise, Aaron should pick the distance
a
that maximizes the
fraction
of the circle of radius
a
located within a distance of |
r
−
e
| of the flag.
Maximizing the fraction of the circle of radius